In [141]:
import numpy as np 
import pandas as pd
import pickle
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

In [142]:
file_name = "train_proc_med.csv"
model_file = "dtMed.sav"

In [143]:
#df = pd.read_csv(file_name,nrows = 5000)
df = pd.read_csv(file_name)
df.head()

,conds,dewptm,fog,hail,hum,pressurem,rain,snow,tempm,thunder,...,North,SE,SSE,SSW,SW,South,Variable,WNW,WSW,West
0,Smoke,9.0,0,0,27.0,1010.0,0,0,30.0,0,...,0,0,0,0,0,0,0,0,0,1
1,Smoke,10.0,0,0,32.0,1017.0,0,0,28.0,0,...,1,0,0,0,0,0,0,0,0,0
2,Smoke,11.0,0,0,44.0,1017.0,0,0,24.0,0,...,1,0,0,0,0,0,0,0,0,0
3,Smoke,10.0,0,0,41.0,1010.0,0,0,24.0,0,...,1,0,0,0,0,0,0,0,0,0
4,Smoke,11.0,0,0,47.0,1011.0,0,0,23.0,0,...,1,0,0,0,0,0,0,0,0,0


In [144]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
target_col = [df.columns[0]]
target = df[target_col]
target = encoder.fit_transform(target)

features = df.drop("conds",axis=1)
features.head()


,dewptm,fog,hail,hum,pressurem,rain,snow,tempm,thunder,tornado,...,North,SE,SSE,SSW,SW,South,Variable,WNW,WSW,West
0,9.0,0,0,27.0,1010.0,0,0,30.0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,10.0,0,0,32.0,1017.0,0,0,28.0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,11.0,0,0,44.0,1017.0,0,0,24.0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,10.0,0,0,41.0,1010.0,0,0,24.0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,11.0,0,0,47.0,1011.0,0,0,23.0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [145]:

scaler = StandardScaler()
features = scaler.fit_transform(features)

features.shape

(5000, 32)

In [146]:
from sklearn.decomposition import PCA

components = 32

pca = PCA(n_components= components )
principalComponents = pca.fit_transform(features)
evSum = np.sum(pca.explained_variance_)
per = lambda X: (X/evSum)*100
perPca = per(pca.explained_variance_)
perPca = np.round_(perPca,2)
perPca[::-1].sort()

print(perPca)


[1.31e+01 8.76e+00 6.33e+00 5.66e+00 4.38e+00 4.04e+00 3.93e+00 3.78e+00
 3.73e+00 3.69e+00 3.65e+00 3.64e+00 3.63e+00 3.61e+00 3.61e+00 3.59e+00
 3.58e+00 3.53e+00 3.22e+00 2.99e+00 2.71e+00 2.14e+00 1.49e+00 1.08e+00
 9.00e-02 5.00e-02 1.00e-02 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00]


In [147]:
li = []
for i in range(components):
    temp = "pc"+str(i+1)
    li.append(temp)

    
principalDf = pd.DataFrame(data = principalComponents, columns = li)
principalDf.head()

,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10,...,pc23,pc24,pc25,pc26,pc27,pc28,pc29,pc30,pc31,pc32
0,1.984882,-2.807314,-0.546725,0.572469,0.624277,1.197994,-0.211122,-0.281540,-0.108592,-0.139277,...,-0.081763,-0.014949,-0.385916,0.184284,0.052817,-5.229413e-15,-2.859910e-15,3.182029e-16,6.042855e-16,-1.585934e-17
1,-1.267266,-1.827608,-2.061861,-0.199853,0.006253,-0.020873,0.002311,-0.143946,-0.012378,-0.051081,...,0.021688,-0.909055,-0.370005,0.127375,-0.008203,-9.473173e-16,-4.154168e-16,-1.282626e-17,5.471190e-18,-5.275532e-17
2,-1.587911,-1.616580,-1.864402,-0.089686,-0.093076,0.074455,-0.004905,-0.126762,-0.065371,-0.026657,...,0.014267,-0.507575,-0.356064,0.032300,-0.006182,2.429284e-16,6.936815e-15,9.023106e-15,5.310827e-16,-5.163930e-18
3,-1.571443,-1.723639,-1.849320,-0.090468,-0.038518,0.037438,-0.007062,-0.120911,-0.048157,-0.029645,...,0.009768,-0.551441,-0.330976,0.033158,-0.006317,-1.082464e-15,-5.087922e-16,3.848315e-17,1.708746e-16,4.245798e-17
4,-1.899658,-1.551342,-1.744920,-0.022405,-0.156211,0.090860,-0.011334,-0.126367,-0.072487,-0.043150,...,0.068582,-0.766927,-0.328838,0.038921,-0.004319,1.178489e-16,1.044772e-16,-3.153232e-17,-4.792364e-17,-1.678181e-17


In [148]:
'''from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier

feature_approx = RBFSampler(gamma=0.01, random_state=100)
principalDf = feature_approx.fit_transform(principalDf)
model = SGDClassifier(max_iter=1000)'''

'from sklearn.kernel_approximation import RBFSampler\nfrom sklearn.linear_model import SGDClassifier\n\nfeature_approx = RBFSampler(gamma=0.01, random_state=100)\nprincipalDf = feature_approx.fit_transform(principalDf)\nmodel = SGDClassifier(max_iter=1000)'

In [149]:
X_train, X_test, y_train, y_test = train_test_split(principalDf,target,test_size = 0.2)

In [150]:
'''from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors = 50,weights='distance')'''


'''from sklearn.tree import DecisionTreeClassifier
#model = DecisionTreeClassifier(criterion="entropy")
model = DecisionTreeClassifier()'''

'''from sklearn.linear_model import LogisticRegression
model = LogisticRegression()'''

from sklearn.svm import SVC
SVC_gamma = 0.01
SVC_C = 300
model = SVC(kernel='rbf',gamma=SVC_gamma,C=SVC_C)


model.fit(X_train,y_train)

SVC(C=300, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [151]:
pickle.dump(model, open(model_file, 'wb'))

In [152]:
X_test.shape

(1000, 32)

In [153]:
y_pred = model.predict(X_test)

In [154]:
y_pred.shape

(1000,)

In [155]:
metrics.accuracy_score(y_test,y_pred)

0.807

In [156]:
print(y_test[0:5])

[12  3 17 16 17]


In [157]:
print(y_pred[0:5])

[ 3  3  0 16 17]
